# Objet : Test opendata des données de qualité de l'air

## Objectif

- valider sur des cas réels l'outil de traitement des "listes indexées"
- identifier les apports que pourraient avoir ce type d'outil

## Résultats
- interêt d'utiliser les outils Pandas
- à compléter

## Usages possibles 
- à compléter

## Autres points
- chargement sur MongoDB à tester
- Outil de requète à tester

données utilisées : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2022/

------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [1]:
from pprint import pprint
from collections import Counter
from time import time
import csv
from util import util
from observation import Ilist, Iindex
from copy import copy
import pandas as pd

chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/air/data_lcsqa/'

In [2]:

data = []
t0 = time()
nb_fichiers = 1
annee = 2022
mois = 1
jour = 1
for i in range(nb_fichiers):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour+i, '02d') +'.csv'
    data.append(pd.read_csv(file, sep=';'))
data2 = pd.concat(data, ignore_index=True, join='inner').astype('category')
data2.pop('valeur brute')
print('data2 : \n', len(data2), '\n', list(data2), '\n', time()-t0)


data2 : 
 49392 
 ['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', 'Réglementaire', "type d'évaluation", 'procédure de mesure', 'type de valeur', 'valeur', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité'] 
 7.683923244476318


----
## initialisation de l'objet Ilist
- l'initialisation pourrait être automatisée à partir du fichier csv
- identification de 64 775 valeurs différentes sur un total de 11 163 x 49 valeurs ("taux d'unicité" de 12%)
- la taille minimale serait de 1,4 Mo (données csv "quotées") pour un maximum de 9,6 Mo (données csv "quotées")

In [4]:
t0=time()
idxs2 = Ilist.obj(data2)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs2), len(idxs2.idxlen), sum(idxs2.idxlen), time()-t0)
#idxs2.setvar('valeur')
idxs3 = Ilist(idxs2)
print(idxs3 == idxs2)

idxs (len, lenlidx, sumcodec) :  49392 22 3344 3.465939521789551
True


In [12]:
t0=time()
print(idxs2.category, '\n')
print(idxs2.tree(), '\n')
print(idxs2.tree(mode='distance'))
print(time()-t0)

{'Date de début': 'secondary', 'Date de fin': 'coupled', 'Organisme': 'secondary', 'code zas': 'secondary', 'Zas': 'coupled', 'code site': 'secondary', 'nom site': 'secondary', "type d'implantation": 'secondary', 'Polluant': 'secondary', "type d'influence": 'secondary', 'discriminant': 'secondary', 'Réglementaire': 'unique', "type d'évaluation": 'secondary', 'procédure de mesure': 'secondary', 'type de valeur': 'unique', 'valeur': 'secondary', 'unité de mesure': 'secondary', 'taux de saisie': 'unique', 'couverture temporelle': 'unique', 'couverture de données': 'unique', 'code qualité': 'secondary', 'validité': 'secondary'} 

-1: root-derived (49392)
   0 : Date de début (24)
      1 : Date de fin (24)
   5 : code site (532)
      2 : Organisme (18)
      3 : code zas (70)
         4 : Zas (70)
      7 : type d implantation (5)
   6 : nom site (532)
   8 : Polluant (9)
   9 : type d influence (3)
   10: discriminant (26)
   11: Réglementaire (1)
   12: type d évaluation (4)
   13: proc

## formats de base

In [8]:
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='full')
fullsize = len(js)
print('fullsize', len(js), time()-t0)

fullsize 14395519 2.608529567718506


In [9]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 49392 12.371837854385376
controle égalité : True 0.3399074077606201


In [10]:
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='nokeys')
minsize = len(js)
print('minsize', len(js), time()-t0)

t0=time()
js = idxs2.to_obj(encoded=True, encode_format='cbor', modecodec='nokeys')
print('mincborsize', len(js), time()-t0)


minsize 34210 0.010066986083984375
mincborsize 32923 0.0


----
## format default
- 

In [25]:
champ = idxs2.nindex
'''champ('valeur').tostdcodec(inplace=True)
idxs2.setcanonorder()
pprint(idxs2.category)'''
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='default')
defaultsize = len(js)
print('defaultsize : ', len(js), time()-t0, '\n')
print('indicator default : ', idxs2.indicator(fullsize, defaultsize), '\n')

t0=time()
pprint(champ('code site').couplinginfos(champ('Date de début')))
print('\n', idxs2.tree(mode='diff'))
print('\nanalyse : ', time()-t0)

defaultsize :  4160319 0.24941802024841309 

indicator default :  {'total values': 1136016, 'mean size': 12.672, 'unique values': 50802, 'mean coding size': 3.24, 'unicity level': 0.045, 'optimize level': 0.289, 'object lightness': 0.256, 'maxgain': 0.955, 'gain': 0.711} 

{'diff': 508,
 'dist': 12768,
 'distance': 12744,
 'distmax': 12768,
 'distmin': 532,
 'distrate': 1.0,
 'disttomax': 0,
 'disttomin': 12236,
 'rate': 1.0,
 'typecoupl': 'crossed'}

 -1: root-diff (49392)
   0 : valeur (1.00e+00 - 49392)
      1 : Date de début (0.00e+00 - 24)
         15: Date de fin (0.00e+00 - 24)
      2 : Organisme (0.00e+00 - 18)
      3 : code zas (0.00e+00 - 70)
         16: Zas (0.00e+00 - 70)
      4 : code site (0.00e+00 - 532)
      5 : nom site (0.00e+00 - 532)
      6 : type d implantation (0.00e+00 - 5)
      7 : Polluant (0.00e+00 - 9)
      8 : type d influence (0.00e+00 - 3)
      9 : discriminant (0.00e+00 - 26)
      10: type d évaluation (0.00e+00 - 4)
      11: procédure de mesu

In [26]:
idxs4 = copy(idxs2)
champ = idxs4.nindex

In [47]:
print(idxs2.lname)

['valeur', 'Date de début', 'Organisme', 'code zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', "type d'évaluation", 'procédure de mesure', 'unité de mesure', 'code qualité', 'validité', 'Date de fin', 'Zas', 'Réglementaire', 'type de valeur', 'taux de saisie', 'couverture temporelle', 'couverture de données']


In [45]:
print(idxs2.analysis.getmatrix(['Polluant', 'unité de mesure']))
notcoupl = champ('Polluant').coupling(champ('unité de mesure'), derived=True)
print('nombre de non couplés : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = [(champ('Polluant')[i], champ('unité de mesure')[i]) for i in notcoupl[:2000]]
print('liste')
#liste = []
#for i in range(6000): liste.append((champ('Polluant')[notcoupl[i]], champ('unité de mesure')[notcoupl[i]]))
pprint(set(liste), width=120)

AttributeError: 'Ilist' object has no attribute 'indexname'

In [11]:
print(idxs2.analysis.getmatrix(['code site', 'nom site']))
notcoupl = champ('code site').coupling(champ('nom site'), derived=False)
print('nombre de non couplés : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = []
for i in range(len(notcoupl)): liste.append((champ('code site')[notcoupl[i]], champ('nom site')[notcoupl[i]]))
pprint(set(liste), width=120)

{'lencoupling': 534, 'distrate': 7.079846508927687e-06, 'disttomin': 2, 'disttomax': 282490, 'distmin': 532, 'distmax': 283024, 'diff': 0, 'typecoupl': 'link'}
nombre de non couplés :  168

liste des premières incohérences : 
{('FR19053', 'QUIMPER ZOLA'), ('FR19053', 'Quimper Zola'), ('FR19007', 'HALLES'), ('FR19007', 'Rennes Les Halles')}


In [12]:
print(idxs2.analysis.getmatrix(['code site', 'nom site']))
champ = idxs2.nindex 
notcoupl = champ('nom site').coupling(champ('code site'), derived=False)
print('nombre de non couplés : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = []
for i in range(len(notcoupl)): liste.append((champ('code site')[notcoupl[i]], champ('nom site')[notcoupl[i]]))
pprint(set(liste), width=120)

{'lencoupling': 534, 'distrate': 7.079846508927687e-06, 'disttomin': 2, 'disttomax': 282490, 'distmin': 532, 'distmax': 283024, 'diff': 0, 'typecoupl': 'link'}
nombre de non couplés :  384

liste des premières incohérences : 
{('FR23078', 'SAINT EXUPERY'), ('FR20048', 'SAINT EXUPERY'), ('FR33101', 'PASTEUR'), ('FR23004', 'PASTEUR')}


In [13]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 49392 0.6975090503692627
True
controle égalité : True 0.31943821907043457


----
## Format optimisé
- 

In [44]:
idxs4.reindex()
idxs4.coupling(param='distance', level=500)
print(idxs4.tree())
t0=time()
js = idxs4.to_obj(modecodec='optimize', encoded=True)
optimizesize = len(js)
print('optimizesize : ', optimizesize, time()-t0, '\n')
print('indicator optimize : ', idxs2.indicator(fullsize, optimizesize), '\n')

t0=time()
js = idxs4.to_obj(encoded=True, modecodec='optimize', encode_format='cbor')
cborsize = len(js)
print('cborsize : ', cborsize, time()-t0, '\n')
print('indicator cbor : ', idxs2.indicator(fullsize, cborsize))

-1: root-derived (49392)
   0 : valeur (1956)
   1 : Date de début (24)
      15: Date de fin (24)
   4 : code site (2616)
      3 : code zas (2143)
         2 : Organisme (18)
         11: procédure de mesure (1299)
            7 : Polluant (310)
               6 : type d implantation (101)
                  10: type d évaluation (49)
                     12: unité de mesure (21)
                        8 : type d influence (9)
                           13: code qualité (3)
                              14: validité (2)
            9 : discriminant (26)
         16: Zas (70)
      5 : nom site (2616)
   17: Réglementaire (1)
   18: type de valeur (1)
   19: taux de saisie (1)
   20: couverture temporelle (1)
   21: couverture de données (1)
optimizesize :  932595 0.3838505744934082 

indicator optimize :  {'total values': 1136016, 'mean size': 12.672, 'unique values': 50802, 'mean coding size': 0.266, 'unicity level': 0.045, 'optimize level': 0.065, 'object lightness': 0.021, 'maxgai

In [15]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 49392 0.6772210597991943
True
controle égalité : True 0.33097004890441895


----
## Format BD
- 

In [16]:
t0=time()
js = idxs2.to_obj(modecodec='dict', encoded=True)
dictsize = len(js)
print('dictsize : ', dictsize, time()-t0, '\n')
print('indicator dict : ', idxs2.indicator(fullsize, dictsize), '\n')

dictsize :  7471841 0.9489867687225342 

indicator dict :  {'init values': 1086624, 'mean size': 13.248, 'unique values': 1413, 'mean coding size': 6.868, 'unicity level': 0.001, 'object lightness': 0.518, 'gain': 0.481} 



In [17]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 49392 0.9998903274536133
True
controle égalité : True 0.34736156463623047
